from this data,I am trying to build an over powered team suitable for my playing style and my budget as I will try to understand some of players stats and rating  

In [ ]:
#Importing the Libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#Turning off the warnings
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as stats

In [ ]:
#Imprting the Date
fifa = pd.read_csv('../input/fifa-21-complete-player-dataset/fifa21_male2.csv')

In [ ]:
fifa.shape

In [ ]:
#to display all the columns 
pd.set_option('display.max_columns',170)

In [ ]:
fifa.head()

Droping all the columns I think it is not usefull to my analyze 

In [ ]:
fifa.columns

In [ ]:
fifa.drop(['ID','Position','Club Logo','Player Photo','Flag Photo','Team & Contract','Team & Contract'],axis=1,inplace=True)
fifa.drop(['Joined','Loan Date End','Value','Wage','Release Clause','Contract'],axis=1,inplace=True)
fifa.drop(['IR','LS','ST','RS','LW','LF','CF','RF','RW','LAM','CAM','RAM','LAM'],axis=1,inplace=True)
fifa.drop(['LM','LCM','CM','RCM','RM','LWB','LDM','CDM','RDM','RWB'],axis=1,inplace=True)
fifa.drop(['LB','LCB','CB','RCB','RB','GK','Gender','Hits'],axis=1,inplace=True)

I am interested in only gold players so I will filter out all the players which are not by taking all players with a rating above75

In [ ]:
fifa=fifa[(fifa['OVA']>75)].reset_index(drop=True)

In [ ]:
fifa.shape

now we have only 1743 gold players out of more than 17000, meaning gold players present around 10% of all players.
Next I will check the info to check the data types if it is correct

In [ ]:
fifa.info()

it shows that Highet and Weight are not in the correct format so is weak foot and skill movement so I will check the data head to see why!

In [ ]:
fifa.head(2)

Weight and Height are in pounds feets and which contain a nun numeric values as describtion for unit which I do not understand so I will conver it to KGs and CMs, weak foot and skill move are containg a star semple and all should be clean and correct by the next function

In [ ]:
def cleaning(x):
    try:
        #Height
        x = x.replace("'",".")
        x = x.replace('"',"")
        #Weight
        x=x.replace("lbs",'')
        # W/F and SM
        x=x.replace("★",'')
        if ((".") in x):
            x = x.split(".")
            x = round((int(x[0])*12+int(x[1])) * 2.54, 1)
        return int(x)
    
    except:
        return int(x)
fifa['Height']= fifa['Height'].apply(cleaning)
#clean the weight and conver it to KG as integers
fifa['Weight'] = ((fifa['Weight'].apply(cleaning))*0.453592).astype(int)
fifa['W/F']= fifa['W/F'].apply(cleaning)
fifa['SM'] = fifa['SM'].apply(cleaning)

Checking again the columns info to check the corrections and the missing values

In [ ]:
fifa.info()

my guess for the missing values that they return to Goalkeeprs so I will check it out

In [ ]:
fifa[fifa['BP']=='GK'].head()

my guess was not correct since some of the values ara missing while other have numeric so I am going to fill the missing data for all columns by the function below

In [ ]:
def missing_value (name_of_the_column):
    
    fifa[name_of_the_column] = fifa.groupby(['OVA','BP'])[name_of_the_column].apply(lambda x: x.fillna(x.mean()))
    fifa[name_of_the_column] = fifa[name_of_the_column].fillna(int(fifa[name_of_the_column].mean()))

for i in ['Balance','Jumping','Volleys','Curve','Agility','Interceptions','Positioning','Vision','Composure','Sliding Tackle']:
    missing_value(i)


similer function but it will take the most frequent object based on both rating and position 

In [ ]:

def missing_object (name_of_the_column):
    
    fifa[name_of_the_column] = fifa.groupby(['OVA','BP'])[name_of_the_column].apply(lambda x: x.fillna(x.value_counts().idxmax()))
    fifa[name_of_the_column] = fifa[name_of_the_column].fillna(fifa[name_of_the_column].value_counts().idxmax())
for j in ['A/W','D/W']:
    missing_object(j)


checking for null values expecting it will be five which are in the club name and I don't belive it worth fixing

In [ ]:
fifa.isnull().sum().sum()

now I will create multiple data frames based on each position

In [ ]:
positions=[]
for i in fifa.BP.unique():
    positions.append(i)
    vars()[i]=fifa[fifa['BP']==i].reset_index(drop=True)


print(positions)

to explain the above loop the vars() function will convert the assigned string to a variable that will equal the data frame where the position equal the string, to check the result I will check the head of the position LWB data frame

In [ ]:
LWB.head(2)

now I will create six more data frames that will contain attacking as ST and CF, midfield containg CM, CDM and CAM , defenders containg CB,RB,RWB,LB and LWB ,leftfield containg LM and LW, rightfield containg RM and RW , and finally sides containg right and left sides

In [ ]:
attacking=pd.concat([ST, CF], axis = 0).sort_values(by=['OVA'], ascending=False).reset_index(drop=True)
defending=pd.concat([RWB,RB,CB,LB,LWB], axis = 0).sort_values(by=['OVA'], ascending=False).reset_index(drop=True)
midfield=pd.concat([CM,CAM,CDM], axis = 0).sort_values(by=['OVA'], ascending=False).reset_index(drop=True)

rightfield=pd.concat([RM,RW], axis = 0).sort_values(by=['OVA'], ascending=False).reset_index(drop=True)
leftfield=pd.concat([LM,LW], axis = 0).sort_values(by=['OVA'], ascending=False).reset_index(drop=True)

sides=pd.concat([RM,RW,LM,LW], axis = 0).sort_values(by=['OVA'], ascending=False).reset_index(drop=True)

to chek if the new data frames in the correct format I will count how many players in each positions in sides then will check its head

In [ ]:
sides['BP'].value_counts() 

In [ ]:
sides.head(2)

In [ ]:
fifa.select_dtypes([np.number]).hist(figsize=(24,24))
plt.show()

it is interisting to see most of the data is either having a normal distribution or left skewness

at the moment I am intrested to check the central limit theorey,it states that, when independent random variables are added, their sum tends toward a normal distribution (commonly known as a bell curve), even if the original variables themselves are not normally distributed. In more precise terms, given certain conditions, the arithmetic mean of a sufficiently large number of iterates of independent random variables — each with a well-defined (finite) expected value and finite variance — will be approximately normally distributed, regardless of the underlying distribution. 

In [ ]:
def sampler(population, n=30, k=500):
    sample_means = []
    for i in range(k):
        sample = np.random.choice(population, size=n, replace=True)
        sample_means.append(np.mean(sample))
    return sample_means


m,n=0 ,0
fig, ax = plt.subplots(4,2,figsize=(10,20))
for i in ['Height','Acceleration','Reactions','Jumping','Stamina','Aggression','Interceptions']:
    sns.distplot(sampler(fifa[i]), bins=100, kde=True,ax=ax[n][m]).set_title(i)
    m+=1
    if m==2:
        n+=1
        m=0


Below I will create a function that find players with a the probability of being among the top 5% of the distribution 
The method based on the Z-Score test  and it is used to find the outlayers among the destribution,The z-score of an observation quantifies how many standard deviations the observation is away from the population mean, 
To find the value of Z, z=((x-mean)/std) and it is probabilty and doing the inverse to check.


In [ ]:
def OP(dataframe, column,top=0.05):
    P=stats.norm.ppf(1-top)
    z=stats.zscore(dataframe[column])
    a,b,c,d,e,f=[],[],[],[],[],[]
    for x,i in  enumerate(z):
#this functi
        if z[x]>P:
            a.append(dataframe['Name'][x])
            b.append(dataframe['BP'][x])
            c.append(z[x])
            d.append(dataframe['Club'][x])
            e.append(dataframe['OVA'][x])
            f.append(dataframe['Nationality'][x])
            
    outstanding=pd.DataFrame()
    outstanding['name'] =a
    outstanding['club']=d
    outstanding['nationality']=f
    outstanding['position']=b
    outstanding['rating']=e
    outstanding['z value']=c
    outstanding=outstanding.sort_values(by=['z value'], ascending=False).reset_index(drop=True)
    
    del outstanding['z value']
    return outstanding

To get benefit from the last two functions, I am looking for  players with Gullit style meaning midfield players with high total stats

In [ ]:
OP(midfield,'Total Stats')

the relation between midfield players and there rating is never clear to me so I will look for an answer by using heat correlation map 

In [ ]:
# the .corr() will return a corrletion matrix between all fields. I am intrested in rating effect so I will loc its row

corr = midfield.corr().loc[:,['OVA']]

fig, ax = plt.subplots(figsize=(5,15))

ax = sns.heatmap(corr.sort_values(by=['OVA'],ascending=False), ax=ax,annot=True,cbar=True,cmap="Greens")
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10)

plt.show()

The result shows it is highly related to short passing, ball control and reactions

Now I will create new field in attacking dataframe that will find the sum of columns that present the stats I am more concern in my strickers with penelty value based on the stat important to me

In [ ]:
attacking.head(1)

In [ ]:
A_stat=['Finishing','Heading Accuracy','Short Passing','Dribbling','Acceleration','Sprint Speed','Agility','Reactions','Balance',
      'Shot Power','Stamina','Strength','Long Shots','Positioning','Vision','Composure','W/F','SM']
A_panelty=[0.9,0.3,0.5,0.7,1.8,4,2,0.9,0.8,
         1.8,0.75,0.6,0.5,3,0.7,0.75,20*2,20*3]

S_stat=A_stat
S_stat.append('Crossing')
S_stat.append('Curve')
S_panelty=[0.6,0.2,3,3,2,2,2,2,2,
          1,3,2,1,2,3,2,20*2,20*3,2,1.5]

attacking['Important STAT']=0

for m in range(len(A_panelty)):
    attacking['Important STAT']+=A_panelty[m]*attacking[A_stat[m]]

sides['Important STAT']=0
for n in range(len(S_panelty)):
    sides['Important STAT']+=S_panelty[n]*sides[S_stat[n]]

In [ ]:
OP(attacking,'Important STAT',0.1)

In [ ]:
OP(sides,'Important STAT')

The panelty it is not important as the value by it self but its value out of all other columns value since z-score value will be calculated
Now the important stats for definding will be diffrent so I will repeat the steps above
To me, I put my RB or LB on the instrecture stay back so thier passes are not important

In [ ]:
defending.head(1)

In [ ]:
D_stat=['Height','Acceleration','Sprint Speed','Reactions','Jumping','Stamina','Strength','Aggression','Interceptions',
       'Composure','Marking','Standing Tackle','Sliding Tackle']
D_panelty=[1.5,4,4,1,1,2,1,1.5,2,
          1,3,4,1]

defending['Important STAT']=0

for j in range(len(D_panelty)):
    defending['Important STAT']+=D_panelty[j]*defending[D_stat[j]]
   
OP(defending,'Important STAT')

I want to check if thre is a relation between the weight and highet of the defender and his stats

In [ ]:
defender=defending.copy()
del defender['Important STAT']

corr = defender.corr().loc[:,['Height','Weight']]

fig, ax = plt.subplots(figsize=(5,15))

ax = sns.heatmap(corr.sort_values(by=['Height'],ascending=False), ax=ax,annot=True,cbar=True,cmap="YlGnBu")
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10)

plt.show()

the graph shows both of them effect heading accurecy, DRI , Balance, Agility, Strength and movement
 but what is intresting to me is highet have more impact on Balance than weight, and height effects positining!
 

I am more intrested in defenders from Bundlisga so I will be able to have Nuer as my GoalKeeper, the code below will tackel this problem

In [ ]:
Bundesliga=["1. FC Köln", "1. FSV Mainz 05", "DSC Arminia Bielefeld", "	Borussia Dortmund", "FC Augsburg", "FC Bayern München", "FC Schalke 04",
    "Eintracht Frankfurt", "Hertha BSC", "Bayer 04 Leverkusen", "Borussia Mönchengladbach", "RB Leipzig","SC Freiburg",
    "TSG 1899 Hoffenheim","1. FC Union Berlin", "VfB Stuttgart", "VfL Wolfsburg", "SV Werder Bremen"]


OP(defending[defending['Club'].isin(Bundesliga)].reset_index(drop=True),'Important STAT',0.1)

While going throw the results of Over Power players I have  seen some players assigned to wrong positions such as Ndidi,
anyhow I am confenced there are more players assigned to the wrong position, so I will check again for  middle players with Gullit style 

In [ ]:
OP(fifa,'Total Stats',0.1)

The result shows that Griezmann and Tells could be realy good midfield players and E.can is a middle player who was assigned to CB position by mistake

after seeing the results, I have decided to use Rashford, Mertinz, Di Maria, Bruno Fernandes , Vidal, Naingolan, N. Mukiele, L.Hernández,L. Klostermann B,Alex Telles,Neuer

As I have chosse my defenders from Bundesliga because I like Neuer, it is time to find more details about GoalKeepres

In [ ]:
GK.describe()

In [ ]:
X=GK.copy()

del X["Name"]
del X['Club']
del X['Nationality']
del X['D/W']
del X['A/W']
del X['foot']
del X['BP']

gk=X.copy()

X = pd.get_dummies(X, drop_first=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

SS = StandardScaler()
X = pd.DataFrame(SS.fit_transform(X),columns=X.columns)

Assuming there is four types of goal keeprs I will check which players are more similer to Neuer by using Kmeans clustring

In [ ]:
# now the similarity of the data will be investigated by applying supervised clustring first and then unsupervised
from sklearn.cluster import KMeans
kmeans=KMeans(n_clusters=4).fit(X)
labels = kmeans.predict(X)
centroids = kmeans.cluster_centers_
GK["Kmeans label"]=labels

gk["Kmeans label"]=labels


In [ ]:
GK[GK['Club']=="FC Bayern München"]

from the dataframe I see Neuer is in cluster three, I will check other goal keeprs in his cluster

In [ ]:
KM=GK[['Name','OVA','Nationality','Club']][GK['Kmeans label']==1].reset_index(drop=True)
KM.head(15)

The dataframe above had all the good goal keeprs  but I need more details so I will Apply DSBCAN to get a better result since the number of clusters is not important

In [ ]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps=5, min_samples=2).fit(X)
clustering.labels_
GK['DSBCAN label']=clustering.labels_
gk['DSBCAN label']=clustering.labels_

I Will check how many clusters where found for goal keepers

In [ ]:
GK['DSBCAN label'].value_counts()

In [ ]:
GK[['Name','OVA','Nationality','Club','DSBCAN label']][GK['DSBCAN label']>=0].reset_index(drop=True)

Neuer Did not apper in the small clusters so I will repeat KMeans with 20 clusters 

In [ ]:
kmeans=KMeans(n_clusters=20).fit(X)
labels = kmeans.predict(X)
centroids = kmeans.cluster_centers_
GK["Kmeans label"]=labels

gk["Kmeans label"]=labels


In [ ]:
GK[['Name',"Kmeans label"]][GK['Club']=="FC Bayern München"]

I See now Neuer Is in cluster 19 so I Will check who else on it

In [ ]:
GK[['Name','OVA','Nationality','Club']][GK["Kmeans label"]==8].reset_index(drop=True)

Now that does make sense to me since all these goal keeprs are very good

finally I want to Know what effect these clusters

In [ ]:
del GK['DSBCAN label']
GK["Kmeans label"]=GK["Kmeans label"].astype(str)

In [ ]:
GK_Kmeans=pd.concat([GK,pd.get_dummies(GK["Kmeans label"])], axis = 1)

In [ ]:
GK_Kmeans.head(2)

In [ ]:
corr = GK_Kmeans.corr().loc[['8'],:].T

fig, ax = plt.subplots(figsize=(7,17))

ax = sns.heatmap(corr.sort_values(by=['8'],ascending=False), ax=ax,annot=True,cbar=True,cmap="seismic")
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10)

plt.show()

it is intresting to see passing, skill and physical having an impact on cluster 8 or as I call it Neuer cluster but not highet! so I will check the effect of highet

In [ ]:
goalkeeper = GK[['GK Diving','GK Handling','GK Kicking','GK Positioning','GK Reflexes','Height']]
corr=goalkeeper.corr().loc[:,['Height']].T
del corr['Height']
fig, ax = plt.subplots(figsize=(10,2))

ax = sns.heatmap(corr.sort_values(by=['Height'],axis=1,ascending=False), ax=ax,annot=True,cbar=True,cmap="rocket_r")
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10)

plt.show()

it almost has no impact!

The final thing I want to test is Benford’s law (also called the first digit law) states that the leading digits in a collection of data set are probably going to be small. For example, most numbers in a set (about 30%) will have a leading digit of 1, when the expected probability is 11.1% (i.e. one out of nine digits). This is followed by about 17.5% starting with a number 2. This is an unexpected phenomenon; If all leading numbers (0 through 9) had equal probability, each would occur 11.1% of the time. To put it simply, Benford’s law is a probability distribution for the likelihood of the first digit in a set of numbers

In [ ]:
Benford=pd.DataFrame()
cols=['fifa','midfield','sides','attacking','defending']
for column in cols:
    Benford[column]=vars()[column].sum(axis =1, skipna = True).astype(int).astype(str).str[0]


Benford.head()

In [ ]:
m,n=0 ,0
fig, ax = plt.subplots(2,3,figsize=(10,10))

for column in cols:
    Benford[column].value_counts().sort_index().plot(kind='bar',figsize=(12,6),title=column,ax=ax[n][m])
    m+=1
    if m==3:
        n+=1
        m=0

The result show that the theorey is not applicable may be because the data is not randomly selected